In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D

# change tensorflow default behavior (where it uses all of the memory at the outset)
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

from sklearn.model_selection import train_test_split

import numpy as np
from matplotlib import pyplot as plt
import random
from datetime import datetime as dt
import os
import cv2

# 1. Data Preparation
### 1.1 Data Extraction

In [ ]:
def load_data(img_size, categories):
    data = []
    data_dir = './resources/images'
    
    for category in categories:
        path = os.path.join(data_dir, category)

        for i, img in enumerate(os.listdir(path)):
            if i >= 500:
                break

            try:
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                img_arr = cv2.resize(img_arr, (img_size, img_size))
                data.append([img_arr, categories.index(category)])
                
            except Exception as e:
                pass
    random.shuffle(data)
    return data

### 1.2 Input & Target Preparation

In [ ]:
img_size = 80
categories = ['Dog', 'Cat']
data = load_data(img_size, categories=categories)

inputs = []
targets = []
for features, label in data:
    inputs.append(features)
    targets.append(label)

inputs = np.array(inputs).reshape(-1, img_size, img_size, 1) # 1 for grayscale, 3 for color
targets = np.array(targets)

### 1.3 Data Normalization

In [ ]:
inputs = inputs / 255.0 # pixel ranged from 0 - 255

### 1.4 Train & Test Preparation

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(inputs, targets, test_size=0.2, random_state=4)

print('x_train.shape:\t', x_train.shape)
print('y_train.shape:\t', y_train.shape)

# 2. Neural Network
### 2.1 Model Definition

In [ ]:
model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=inputs.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

### 2.2 Train Model

In [ ]:
start_time = dt.now().strftime("%Y-%m-%d %H:%M:%S.%f")
history = model.fit(x_train, y_train, epochs=50, validation_data=(x_test, y_test))
end_time = dt.now().strftime("%Y-%m-%d %H:%M:%S.%f")

### 2.3 Result Prediction

In [ ]:
print(f'Train Start:\t{start_time}')
print(f'Train End:\t{end_time}')

results = model.predict(x_test)

# 3. Result Visualization

In [ ]:
for i, arr in enumerate(x_test[:10]):
    label = categories[y_test[i]]
    predict = categories[int(round(results[i][0]))]
    
    title_obj = plt.title(f'Labeled: {label} | Predicted: {predict}')
    if (label != predict):
        plt.setp(title_obj, color='r')
    else:
        plt.setp(title_obj, color='g')
    
    plt.imshow(np.array(arr).reshape(-1, img_size, img_size)[0] * 255.0, cmap='gray')
    plt.show()